In [1]:
import pandas as pd
import numpy as np
import operator
import re
from collections import Counter

search_term = 'ziek'
doc = pd.read_csv('sample_dataset.csv', sep=';')

doc.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,ziek src:twitter,2017-09-20 10:11,https://twitter.com/HubertDeMeulder/status/910...,-,post,NaN,14671.0,HubertDeMeulder,14671,4.8,NaN,NaN,#SabineHagedoren: “Op het werk wist niemand da...,twitter,NaN
1,ziek src:twitter,2017-09-20 10:11,https://twitter.com/michielvdbroeck/status/910...,+,comment,4.0,NaN,michielvdbroeck,1492,1.7,NaN,NaN,@heloisesell Haha wtf ziek nice,twitter,NaN
2,ziek src:twitter,2017-09-20 10:10,https://twitter.com/petervalk1/status/91041589...,-,comment,2.0,1088.0,petervalk1,1088,1.8,NaN,NaN,"RT @AcrisiuS322: Van der Laan is ziek gemeld, ...",twitter,NaN
3,ziek src:twitter,2017-09-20 10:09,https://twitter.com/Korneel_Evers/status/91041...,+,comment,644.0,NaN,Korneel_Evers,4411,21.4,NaN,NaN,@_jazzybelle @zoekpostadres @real_Raffie @olaf...,twitter,NaN
4,ziek src:twitter,2017-09-20 10:07,https://twitter.com/sariehimpe/status/91041518...,-,post,NaN,332.0,sariehimpe,332,0.4,NaN,NaN,Iedereen gaat al een week de tijd gehad hebben...,twitter,NaN


# Cleaning data
Here we do several things:
* remove retweets
* convert text to lowercase

In the context of our research a retweet does not add any relevant data. It actually skews the results since a specific phrasing get's repeated more often than it naturally would.

In [2]:
doc['bericht tekst'] = doc['bericht tekst'].fillna('')

In [3]:
doc['bericht tekst'] = doc['bericht tekst'].str.lower()

In [4]:
doc  = doc[~doc['bericht tekst'].str.contains('rt')]
# # doc[~doc['bericht tekst'].str.contains('rt')]
# doc.head()

In [5]:
doc = doc.drop_duplicates()

# Defenition of helper function


In [6]:
re_clean = re.compile(r'(https?://\S+|@\S+)')
re_words = re.compile(r'(\w+-?\w*)')

def clean_text(text: str):
    words = []
    if text:
        text = re_clean.sub(' ', text)
        words = re_words.findall(text)
    return words

In [7]:
all_tweets = pd.read_csv('sample_alltweets.csv', sep=';')
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
0,src:twitter,2017-06-14 23:59,https://twitter.com/evertdev/status/8751105937...,+,comment,9.0,1214.0,evertdev,1214,1.1,NaN,NaN,RT @JaapTielbeke: Mijn stuk over het onverwach...,twitter,NaN
1,src:twitter,2017-06-14 23:59,https://twitter.com/SoBeOnIt/status/8751105897...,NaN,comment,110.0,236.0,SoBeOnIt,236,0.6,NaN,NaN,RT @RSN_RACAILLE: Conozco a muy pocas personas...,twitter,NaN
2,src:twitter,2017-06-14 23:59,https://twitter.com/spartarn/status/8751105927...,NaN,comment,65.0,50.0,spartarn,50,0.6,NaN,NaN,RT @MLBDraftTracker: 36 (1085): @Tigers select...,twitter,NaN
3,src:twitter,2017-06-14 23:59,https://twitter.com/Neraajz/status/87511059080...,NaN,post,NaN,1548.0,Neraajz,1548,2.9,NaN,NaN,Overmorgen examen en volgende week ook en dan ...,twitter,NaN
4,src:twitter,2017-06-14 23:59,https://twitter.com/pepsi_elwin/status/8751105...,NaN,comment,NaN,NaN,pepsi_elwin,250,1.0,NaN,NaN,@Redwiff that probly requires hacking your swi...,twitter,NaN


# Cleaning data
Here we do several things:
* convert messages with no content with to the message `''`
* remove retweets
* convert to lowercase

The conversion is needed since the `clean_text()` function expects a string.

In [8]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].fillna('')

In [9]:
all_tweets['bericht tekst'] = all_tweets['bericht tekst'].str.lower()

In [10]:
all_tweets = all_tweets[~all_tweets['bericht tekst'].str.contains('rt')]
all_tweets.head()

,zoekopdracht,datum,url,sentiment,type,discussielengte,views,auteur,volgers,invloed,GPS breedtegraad,GPS lengtegraad,bericht tekst,type bron,titel
3,src:twitter,2017-06-14 23:59,https://twitter.com/Neraajz/status/87511059080...,NaN,post,NaN,1548.0,Neraajz,1548,2.9,NaN,NaN,overmorgen examen en volgende week ook en dan ...,twitter,NaN
4,src:twitter,2017-06-14 23:59,https://twitter.com/pepsi_elwin/status/8751105...,NaN,comment,NaN,NaN,pepsi_elwin,250,1.0,NaN,NaN,@redwiff that probly requires hacking your swi...,twitter,NaN
6,src:twitter,2017-06-14 23:59,https://twitter.com/KvW_beach/status/875110593...,NaN,post,NaN,71.0,KvW_beach,71,0.6,NaN,NaN,morgen begint het tornooi voor ons in den haag...,twitter,NaN
7,src:twitter,2017-06-14 23:59,https://twitter.com/rene_fiala/status/87511059...,+,comment,NaN,NaN,rene_fiala,29,0.3,NaN,NaN,@kivicod @stoneemlg wtf hahaha 😂,twitter,NaN
9,src:twitter,2017-06-14 23:59,https://twitter.com/5HOfficialMex/status/87511...,NaN,post,3.0,27834.0,5HOfficialMex,27834,1.7,NaN,NaN,normani en el snapchat de @joshleviworld #3 ht...,twitter,NaN


In [14]:
all_tweets['bericht woorden'] = all_tweets['bericht tekst'].map(clean_text)

counter = Counter()

for words in all_tweets['bericht woorden']:
    counter.update(words)
counter.most_common(300)

[('de', 2968),
 ('in', 2469),
 ('is', 2443),
 ('ik', 2419),
 ('een', 2219),
 ('en', 2172),
 ('van', 1699),
 ('het', 1695),
 ('the', 1639),
 ('je', 1542),
 ('op', 1330),
 ('a', 1324),
 ('i', 1324),
 ('dat', 1256),
 ('to', 1182),
 ('met', 1103),
 ('voor', 1086),
 ('niet', 1080),
 ('of', 1015),
 ('s', 968),
 ('you', 899),
 ('te', 819),
 ('maar', 789),
 ('and', 780),
 ('die', 775),
 ('me', 714),
 ('zijn', 675),
 ('it', 673),
 ('ook', 657),
 ('for', 623),
 ('er', 593),
 ('on', 587),
 ('dan', 551),
 ('aan', 542),
 ('wat', 538),
 ('als', 523),
 ('via', 522),
 ('we', 517),
 ('t', 516),
 ('nog', 489),
 ('was', 485),
 ('wel', 481),
 ('zo', 476),
 ('bij', 472),
 ('om', 468),
 ('heb', 443),
 ('mijn', 438),
 ('ben', 437),
 ('over', 435),
 ('2', 428),
 ('nu', 422),
 ('ze', 422),
 ('my', 421),
 ('this', 419),
 ('dit', 419),
 ('that', 412),
 ('1', 367),
 ('kan', 359),
 ('al', 344),
 ('naar', 341),
 ('so', 339),
 ('geen', 327),
 ('haar', 323),
 ('with', 320),
 ('ja', 315),
 ('weer', 310),
 ('m', 310),


In [12]:
common_words = [word[0] for word in counter.most_common(300)]
common_words = set(common_words)
common_words

['de',
 'in',
 'is',
 'ik',
 'een',
 'en',
 'van',
 'het',
 'the',
 'je',
 'op',
 'a',
 'i',
 'dat',
 'to',
 'met',
 'voor',
 'niet',
 'of',
 's',
 'you',
 'te',
 'maar',
 'and',
 'die',
 'me',
 'zijn',
 'it',
 'ook',
 'for',
 'er',
 'on',
 'dan',
 'aan',
 'wat',
 'als',
 'via',
 'we',
 't',
 'nog',
 'was',
 'wel',
 'zo',
 'bij',
 'om',
 'heb',
 'mijn',
 'ben',
 'over',
 '2',
 'nu',
 'ze',
 'my',
 'this',
 'dit',
 'that',
 '1',
 'kan',
 'al',
 'naar',
 'so',
 'geen',
 'haar',
 'with',
 'ja',
 'weer',
 'm',
 '3',
 'echt',
 'uit',
 'be',
 'by',
 'u',
 'no',
 'at',
 'video',
 'your',
 'heeft',
 'not',
 'door',
 'meer',
 'man',
 '0',
 'gaat',
 'leuk',
 'just',
 'n',
 'are',
 'vind',
 'what',
 'toch',
 'omg',
 'deze',
 'do',
 'wil',
 'can',
 'nieuws',
 'moet',
 'goed',
 'have',
 'mij',
 'new',
 'but',
 'one',
 'hoe',
 'like',
 'all',
 'veel',
 'hij',
 'he',
 'love',
 'na',
 'jaar',
 'now',
 'even',
 '5',
 'nl',
 'hebben',
 'want',
 'mensen',
 'as',
 '4',
 'out',
 'dus',
 'vrouw',
 'zou',
 '

In [13]:
blacklisted_words = common_words
blacklisted_words.add(search_term)

AttributeError: 'list' object has no attribute 'add'

In [ ]:
doc['bericht woorden'] = doc['bericht tekst'].map(clean_text)
counter = Counter()

for words in doc['bericht woorden']:
    words = set(words)
    filtered_words = words - blacklisted_words
    counter.update(filtered_words)
    
related_words = counter.most_common(100)
related_words

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
most_related_word = related_words[0][0]
print('Word: {}'.format(most_related_word))
doc[doc['bericht tekst'].str.contains(most_related_word)].filter(items=['bericht tekst', 'auteur', 'type'])

In [ ]:
word = related_words[4][0]
print('Word: {}'.format(word))
doc[doc['bericht tekst'].str.contains(word)].filter(items=['bericht tekst', 'auteur', 'type'])